In [1]:
!pip3 install rake-nltk

In [6]:
!pip3 install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39205 sha256=3fe7156230dd87b675d9c9ec91c6ed5a8afdce3a48f6d2bb93fadc596f59d6e1
  Stored in directory: /Users/kossee/Library/Caches/pip/wheels/81/e4/ee/453d310c148e394b6079fa58044fcf88f59d56b385de3ce41e
Successfully built keybert


In [10]:
!pip3 install yake

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 4.2 MB/s eta 0:00:00a 0:00:01


In [12]:
%%time
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

rake = Rake()
text = """
Shaormerii cu specific arabesc in Brasov
"""
rake.extract_keywords_from_text(text)
keywordList = []
rankedList = rake.get_ranked_phrases_with_scores()
for keyword in rankedList:
    keyword_updated = keyword[1].split()
    keyword_updated_string = " ".join(keyword_updated[:2])
    keywordList.append(keyword_updated_string)
    if(len(keywordList)>9):
        break
print(keywordList)

['shaormerii cu', 'brasov']
CPU times: user 39.6 ms, sys: 24.9 ms, total: 64.5 ms
Wall time: 336 ms


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kossee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kossee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
%%time
import spacy
from collections import Counter
from string import punctuation

nlp = spacy.load("en_core_web_sm")

def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result
new_text = """
Shaormerii cu specific arabesc in Brasov
"""
output = set(get_hotwords(new_text))
most_common_list = Counter(output).most_common(10)
for item in most_common_list:
    print(item[0])

arabesc
specific
brasov
cu
CPU times: user 205 ms, sys: 36.1 ms, total: 241 ms
Wall time: 375 ms


In [14]:
%%time
from keybert import KeyBERT
doc = """
         Shaormerii cu specific arabesc in Brasov"""
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)
print(keywords)

[('shaormerii', 0.4858), ('arabesc', 0.477), ('brasov', 0.4707), ('cu', 0.1933), ('specific', 0.0431)]
CPU times: user 1.51 s, sys: 1.64 s, total: 3.15 s
Wall time: 5.82 s


In [15]:
%%time
import yake
doc = """
         Shaormerii cu specific arabesc in Brasov"""
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(doc)
for kw in keywords:
    print(kw)

('arabesc in Brasov', 0.026233073037508336)
('Shaormerii cu specific', 0.04940384002065631)
('Brasov', 0.08596317751626563)
('specific arabesc', 0.09700399286574239)
('Shaormerii', 0.15831692877998726)
('specific', 0.29736558256021506)
('arabesc', 0.29736558256021506)
CPU times: user 7 ms, sys: 3.78 ms, total: 10.8 ms
Wall time: 11.9 ms
